# Código Python - Trabalho 2

## *Imports* estáticos

In [3]:
import pandas as pd
import numpy as np

from sklearn.pipeline import make_pipeline
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.preprocessing import StandardScaler
#from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error, r2_score

## Declaração de variáveis repetidas

In [4]:
random_state = 42
alpha = 0.05
test_size = 0.3

## 4.1  Análise Exploratória de Dados
### 4.1.1   Leitura do ficheiro

In [5]:
dados = pd.read_csv('../../dados/AIRPOL_data.csv', delimiter=";", header=0, decimal=',')
dados = dados.drop(columns=['Unnamed: 8', 'Unnamed: 9', 'Unnamed: 10', 'Unnamed: 11', 'Unnamed: 12', 'Unnamed: 13', 'Unnamed: 14', 'Unnamed: 15']).rename(columns={"Value":"Premature_Deaths","Outcome":"Disease"})
#dados = dados[dados.Country != 'All Countries']
#dados.sort_values(by=['Value'])

### 4.1.4. Agrupamento dos dados em zonas

In [6]:
westEuDados = dados[dados['Country'].isin(['Austria', 'Belgium', 'France', 'Germany', 'Netherlands', 'Switzerland'])]
eastEuDados = dados[dados['Country'].isin(['Poland', 'Czechia', 'Hungary'])]
soutEuDados = dados[dados['Country'].isin(['Greece', 'Spain', 'Italy', 'Portugal'])]
nortEuDados = dados[dados['Country'].isin(['Sweden', 'Denmark', 'Northern Europe', 'Finland'])]

### 4.3.1 Novo atributo "RespDisease"

In [7]:
dados4regioes = pd.concat([westEuDados, eastEuDados, soutEuDados, nortEuDados], ignore_index=False)
dados4regioes

,Country,NUTS_Code,Air_Pollutant,Disease,Affected_Population,Populated_Area[km2],Air_Pollution_Average[ug/m3],Premature_Deaths
133,Austria,AT333,PM2.5,Chronic obstructive pulmonary disease,39690.0,387.5,5.0,0.0
141,Austria,AT321,PM2.5,Chronic obstructive pulmonary disease,16485.0,232.4,5.2,0.0
146,Austria,AT331,PM2.5,Chronic obstructive pulmonary disease,25349.0,227.4,5.0,0.0
152,Austria,AT321,PM2.5,Diabetes Mellitus,16485.0,232.4,5.2,0.0
164,Austria,AT331,PM2.5,Diabetes Mellitus,25349.0,227.4,5.0,0.0
...,...,...,...,...,...,...,...,...
46798,Sweden,SE213,NO2,Stroke,176161.0,5148.4,1.8,0.0
46799,Sweden,SE213,NO2,Stroke,176161.0,5148.4,1.8,0.0
46802,Sweden,SE313,NO2,Stroke,215667.0,4520.6,1.0,0.0
46803,Sweden,SE322,NO2,Stroke,100258.0,4104.6,0.6,0.0


Este código foi inspirado por esta [thread](https://stackoverflow.com/questions/67420811/a-new-column-in-pandas-which-value-depends-on-other-columns) no stack**overflow**

In [8]:
goalAttrib = 'RespDisease'

def calc_resp_disease(row):
    respDiseases = ['Asthma', 'Chronic obstructive pulmonary disease']
    if row['Disease'] in respDiseases:
        return 1
    else:
        return 0

dados4regioes[goalAttrib] = dados4regioes.apply(calc_resp_disease, axis=1)

dados4regioes

,Country,NUTS_Code,Air_Pollutant,Disease,Affected_Population,Populated_Area[km2],Air_Pollution_Average[ug/m3],Premature_Deaths,RespDisease
133,Austria,AT333,PM2.5,Chronic obstructive pulmonary disease,39690.0,387.5,5.0,0.0,1
141,Austria,AT321,PM2.5,Chronic obstructive pulmonary disease,16485.0,232.4,5.2,0.0,1
146,Austria,AT331,PM2.5,Chronic obstructive pulmonary disease,25349.0,227.4,5.0,0.0,1
152,Austria,AT321,PM2.5,Diabetes Mellitus,16485.0,232.4,5.2,0.0,0
164,Austria,AT331,PM2.5,Diabetes Mellitus,25349.0,227.4,5.0,0.0,0
...,...,...,...,...,...,...,...,...,...
46798,Sweden,SE213,NO2,Stroke,176161.0,5148.4,1.8,0.0,0
46799,Sweden,SE213,NO2,Stroke,176161.0,5148.4,1.8,0.0,0
46802,Sweden,SE313,NO2,Stroke,215667.0,4520.6,1.0,0.0,0
46803,Sweden,SE322,NO2,Stroke,100258.0,4104.6,0.6,0.0,0


### 4.3.2 K-Fold cross validation

#### Preparação dos valores

In [9]:
features = list(dados4regioes.columns[0:9])
numericFeatures = features[4:]
print(numericFeatures)
scaler = StandardScaler()

X = dados4regioes[numericFeatures].drop(columns=[goalAttrib])
y = dados4regioes[goalAttrib]

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=test_size,random_state=random_state)

print("Stratified division of goal attribute:")
print(y_train.value_counts(normalize=True).mul(100).round(1).astype(str)+'%')
print(y_test.value_counts(normalize=True).mul(100).round(1).astype(str)+'%')

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

['Affected_Population', 'Populated_Area[km2]', 'Air_Pollution_Average[ug/m3]', 'Premature_Deaths', 'RespDisease']
Stratified division of goal attribute:
RespDisease
0    62.1%
1    37.9%
Name: proportion, dtype: object
RespDisease
0    62.1%
1    37.9%
Name: proportion, dtype: object


#### Otimização dos parâmetros da Árvore de regressão

In [7]:
results = []
for max_depth in range(2, 11):
    for min_samples_split in [2, 5, 10]:
        for min_samples_leaf in [1, 2, 4]:
            dt = DecisionTreeRegressor(
                max_depth=max_depth,
                min_samples_split=min_samples_split,
                min_samples_leaf=min_samples_leaf,
                random_state=random_state
            )
            dt.fit(X_train_scaled, y_train)
            y_pred = dt.predict(X_test_scaled)

            result = {
                'max_depth': max_depth,
                'min_samples_split': min_samples_split,
                'min_samples_leaf': min_samples_leaf,
                'mse': mean_squared_error(y_test, y_pred),
                'r2': r2_score(y_test, y_pred)
            }
            results.append(result)

resdf = pd.DataFrame(results)
resdf = resdf.sort_values(by='r2', ascending=False)
resdf

,max_depth,min_samples_split,min_samples_leaf,mse,r2
73,10,2,2,0.150446,0.362745
76,10,5,2,0.150603,0.362081
79,10,10,2,0.150824,0.361144
75,10,5,1,0.150995,0.360419
72,10,2,1,0.151041,0.360228
...,...,...,...,...,...
4,2,5,2,0.179558,0.239434
5,2,5,4,0.179558,0.239434
6,2,10,1,0.179558,0.239434
7,2,10,2,0.179558,0.239434


Assim, vemos que a melhor árvore tem os seguintes parâmetros:
* max_depth = 10
* min_samples_split = 2
* min_samples_leaf = 2
#### Otimização do kernel SVM

In [12]:
kernels = ['linear', 'rbf', #'poly'
           ]
results = []

for kernel in kernels:
    # initialize and train SVM
    svm_model = SVR(kernel=kernel)
    rmse_scores = cross_val_score(svm_model, X_train_scaled, y_train, scoring='neg_mean_squared_error', cv=10)
    # store results
    results.append({
        "model": kernel,
        "rmse": np.mean(rmse_scores),
        "std": np.std(rmse_scores)
    })
    print(f"{kernel} done")

resdf = pd.DataFrame(results)
resdf = resdf.sort_values(by='rmse', ascending=True)
resdf

linear done
rbf done


,model,rmse,std
0,linear,-0.255862,0.005808
1,rbf,-0.249676,0.006100


Comparando os valores, chegamos à conclusão que o melhor kernel é o "linear".
(O "poly" não foi mostrado visto que demora demasiado (mais de meia hora com cv=2))

#### Otimização da configuração da rede neuronal

In [8]:
def regressmodelevaluation(name_model, y_test, y_pred):
    mae = round(mean_absolute_error(y_test, y_pred), 3)
    mse = round(mean_squared_error(y_test, y_pred), 3)
    rmse = round(np.sqrt(mse), 3)
    r2 = round(r2_score(y_test, y_pred), 3)
    mape = round(mean_absolute_percentage_error(y_test, y_pred), 3)
    return pd.Series({'Model': name_model, 'mae': mae, 'mse': mse, 'rmse': rmse, 'r2': r2, 'mape': mape})

In [11]:
results = []
for solv in ['lbfgs', 'sgd', 'adam']:
    for nodes1 in range(2,6):
        for nodes2 in range(3,7):
            mlp = MLPRegressor(hidden_layer_sizes=(nodes1, nodes2),
                        activation='tanh',
                        solver=solv,
                        max_iter=1000,
                        learning_rate='adaptive',
                        early_stopping=True,
                        random_state=random_state)
            mlp.fit(X_train_scaled, y_train)

            y_pred = mlp.predict(X_test_scaled)
            result = regressmodelevaluation(f'{solv} {nodes1} {nodes2}', y_test, y_pred)
            results.append(result)
    
resdf = pd.DataFrame(results)
resdf.sort_values(by='r2', ascending=False)

C:\Users\35193\AppData\Roaming\Python\Python312\site-packages\sklearn\neural_network\_multilayer_perceptron.py:546: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
C:\Users\35193\AppData\Roaming\Python\Python312\site-packages\sklearn\neural_network\_multilayer_perceptron.py:546: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
C:\Users\35193\AppData\Roaming\Python\Python312\site-packages\sklearn\neural_network\_multilayer_perceptron.py:546: ConvergenceWarning: lb

,Model,mae,mse,rmse,r2,mape
15,lbfgs 5 6,0.338,0.168,0.410,0.290,7.477506e+14
12,lbfgs 5 3,0.340,0.170,0.412,0.281,7.523330e+14
5,lbfgs 3 4,0.347,0.171,0.414,0.274,7.659550e+14
14,lbfgs 5 5,0.351,0.172,0.415,0.271,7.698458e+14
10,lbfgs 4 5,0.347,0.173,0.416,0.268,7.659608e+14
9,lbfgs 4 4,0.351,0.174,0.417,0.265,7.755926e+14
11,lbfgs 4 6,0.350,0.174,0.417,0.265,7.712404e+14
7,lbfgs 3 6,0.352,0.174,0.417,0.264,7.727748e+14
6,lbfgs 3 5,0.355,0.175,0.418,0.258,7.835596e+14
47,adam 5 6,0.355,0.176,0.420,0.254,7.887316e+14


Com isto, vemos que o melhor modelo é o lbfgs 5 6
#### Otimização da configuração dos K-vizinhos mais próximos

In [ ]:
num_holdouts = 20

results = []

for k in range(1, 51, 2):
    rmse_scores = []
    for _ in range(num_holdouts):
        Xhd_train, Xhd_test, yhd_train, yhd_test = train_test_split(X_train_scaled, y_train, test_size=test_size)

        knn_model = KNeighborsRegressor(n_neighbors=k)
        knn_model.fit(Xhd_train, yhd_train)

        y_pred = knn_model.predict(Xhd_test)

        mse = mean_squared_error(yhd_test, y_pred)
        rmse_scores.append(np.sqrt(mse))

    results.append({
        "k": k,
        "rmse_mean": np.mean(rmse_scores),
        "rmse_std": np.std(rmse_scores)
    })

resdf = pd.DataFrame(results)
resdf = resdf.sort_values(by='rmse_mean', ascending=True)
resdf

Average RMSE:

1 - 0.466 +/- (0.003)
3 - 0.424 +/- (0.003)
5 - 0.424 +/- (0.002)
7 - 0.423 +/- (0.002)
9 - 0.422 +/- (0.003)
11 - 0.421 +/- (0.002)
13 - 0.42 +/- (0.002)
15 - 0.419 +/- (0.002)
17 - 0.419 +/- (0.002)
19 - 0.418 +/- (0.002)
21 - 0.419 +/- (0.002)
23 - 0.419 +/- (0.002)
25 - 0.42 +/- (0.002)
27 - 0.42 +/- (0.002)
29 - 0.419 +/- (0.002)
31 - 0.42 +/- (0.003)
33 - 0.42 +/- (0.002)
35 - 0.419 +/- (0.002)
37 - 0.419 +/- (0.002)
39 - 0.42 +/- (0.002)
41 - 0.42 +/- (0.002)
43 - 0.42 +/- (0.002)
45 - 0.42 +/- (0.002)
47 - 0.42 +/- (0.002)
49 - 0.421 +/- (0.001)


k = 15 é o ideal aqui (?)

#### k-fold cross validation

In [13]:
kfold = KFold(n_splits=5, shuffle=True, random_state=random_state)

models = []
#models.append(('rgr', LinearRegression()))
models.append(('dtr', DecisionTreeRegressor(
        max_depth=10,
        min_samples_split=2,
        min_samples_leaf=2,
        random_state=random_state
    )))
models.append(('net', MLPRegressor(hidden_layer_sizes=(5, 6),
                        activation='tanh',
                        solver='lbfgs',
                        max_iter=1000,
                        learning_rate='adaptive',
                        early_stopping=True,
                        random_state=random_state)))
models.append(('knn', KNeighborsRegressor(n_neighbors=21)))
models.append(('svm', make_pipeline(StandardScaler(),SVR(kernel='linear'))))

lstresults = []

for name, model in models:
    # RMSE
    mse_scores = cross_val_score(model, X, y, cv=kfold, scoring='neg_mean_squared_error')
    rmse_scores = (-mse_scores) ** 0.5

    # MAE
    mae_scores = cross_val_score(model, X, y, cv=kfold, scoring='neg_mean_absolute_error')
    mae_scores = -mae_scores

    # R2
    r2_scores = cross_val_score(model, X, y, cv=kfold, scoring='r2')

    lstresults.append(pd.Series({
        'model': name,
        'mean_RMSE': round(np.mean(rmse_scores), 3),
        'std_RMSE': round(np.std(rmse_scores), 3),
        'mean_MAE': round(np.mean(mae_scores), 3),
        'std_MAE': round(np.std(mae_scores), 3),
        'mean_R2': round(np.mean(r2_scores), 3),
        'std_R2': round(np.std(r2_scores), 3)
    }))
    print(f"Model {name} done")

resdf = pd.DataFrame(lstresults)
resdf.sort_values(by='mean_R2', ascending=False)

Model dtr done
Model net done
Model knn done
Model svm done


,model,mean_RMSE,std_RMSE,mean_MAE,std_MAE,mean_R2,std_R2
0,dtr,0.385,0.002,0.288,0.002,0.371,0.005
2,knn,0.450,0.003,0.387,0.002,0.139,0.005
1,net,0.485,0.002,0.471,0.001,-0.000,0.000
3,svm,0.506,0.006,0.346,0.006,-0.087,0.020
